# 25 - Data Governance z Apache Atlas

Apache Atlas to platforma data governance dla ekosystemu Hadoop. Umozliwia katalogowanie, klasyfikacje i sledzenie lineage danych.

**Tematy:**
- Architektura Apache Atlas: Type System, Entity, Classification, Glossary
- Atlas REST API - zarzadzanie z Pythona
- Rejestrowanie typow i encji (datasets, processes)
- Klasyfikacja danych: PII, confidential, public
- Data lineage: sledzenie transformacji Bronze -> Silver -> Gold
- Glossary biznesowy: definiowanie terminow
- Atlas + Spark Hook: automatyczne sledzenie Spark jobow
- Wyszukiwanie: DSL queries, full-text search

## 1. Architektura Apache Atlas

```
                         +-----------------------------+
                         |       Atlas Web UI          |
                         |    (http://atlas:21000)     |
                         +-------------+---------------+
                                       |
                         +-------------v---------------+
                         |      Atlas REST API         |
                         |   /api/atlas/v2/...         |
                         +-------------+---------------+
                                       |
              +------------------------+------------------------+
              |                        |                        |
    +---------v---------+   +----------v----------+   +---------v---------+
    |   Type System     |   |   Graph Engine      |   |   Index Engine    |
    |                   |   |   (JanusGraph)      |   |   (Solr/ES)       |
    | - TypeDefs        |   |   - Entities        |   |   - Full-text     |
    | - Classifications |   |   - Relationships   |   |   - DSL queries   |
    | - Glossary        |   |   - Lineage graph   |   |                   |
    +-------------------+   +---------------------+   +-------------------+
              |
    +---------v---------+
    |   Hooks / Bridges |
    | - Spark Hook      |
    | - Hive Hook       |
    | - Kafka Bridge    |
    +-------------------+
```

### Kluczowe koncepty:
- **Type System**: definiuje schemat metadanych (jak klasy w OOP)
- **Entity**: instancja typu (konkretny dataset, tabela, kolumna)
- **Classification**: etykieta/tag (PII, Confidential, Public)
- **Glossary**: slownik biznesowy z terminami i kategoriami
- **Lineage**: graf pokazujacy skad dane pochodza i dokad ida

## 2. Setup - polaczenie z Atlas REST API

Atlas udostepnia REST API na porcie 21000. Domyslne dane logowania: `admin`/`admin`.

In [ ]:
import requests
import json
from datetime import datetime

# Atlas REST API configuration
ATLAS_URL = "http://atlas:21000/api/atlas/v2"
ATLAS_AUTH = ("admin", "admin")
HEADERS = {"Content-Type": "application/json", "Accept": "application/json"}


class AtlasClient:
    """Klient do komunikacji z Apache Atlas REST API."""

    def __init__(self, base_url, auth):
        self.base_url = base_url
        self.auth = auth
        self.session = requests.Session()
        self.session.auth = auth
        self.session.headers.update(HEADERS)

    def get(self, endpoint, params=None):
        resp = self.session.get(f"{self.base_url}{endpoint}", params=params)
        resp.raise_for_status()
        return resp.json()

    def post(self, endpoint, data):
        resp = self.session.post(f"{self.base_url}{endpoint}", json=data)
        resp.raise_for_status()
        return resp.json()

    def put(self, endpoint, data):
        resp = self.session.put(f"{self.base_url}{endpoint}", json=data)
        resp.raise_for_status()
        return resp.json()

    def delete(self, endpoint):
        resp = self.session.delete(f"{self.base_url}{endpoint}")
        resp.raise_for_status()
        return resp.status_code


atlas = AtlasClient(ATLAS_URL, ATLAS_AUTH)

# Test polaczenia
try:
    version = atlas.get("/admin/version")
    print(f"Atlas version: {version.get('Version', 'unknown')}")
    print(f"Status: Connected")
except requests.exceptions.ConnectionError:
    print("UWAGA: Atlas nie jest dostepny. Uruchom kontener Atlas.")
    print("Notebook mozna czytac jako material edukacyjny - kod pokazuje wzorce uzycia API.")

## 3. Type System - definiowanie typow metadanych

Atlas Type System jest hierarchiczny - typy dziedzicza po sobie:

| Kategoria | Opis | Przyklady |
|-----------|------|----------|
| **Entity** | Obiekty ze stanem i tozsamoscia | DataSet, Process, hdfs_path, hive_table |
| **Classification** | Etykiety/tagi | PII, Confidential, Public |
| **Struct** | Typy zlozoone (bez tozsamosci) | hive_serde, repl_config |
| **Enum** | Typy wyliczeniowe | file_format, data_quality_level |
| **Relationship** | Relacje miedzy encjami | dataset_process_inputs |

Wbudowane supertypy:
- `DataSet` - dane (tabele, pliki, tematy Kafka)
- `Process` - transformacje (joby Spark, zapytania SQL)
- `Infrastructure` - infrastruktura (klaster, serwer)

In [ ]:
# Przegladanie istniejacych typow
try:
    all_typedefs = atlas.get("/types/typedefs")
    print("Istniejace typy w Atlas:")
    for category in ["entityDefs", "classificationDefs", "enumDefs", "structDefs", "relationshipDefs"]:
        defs = all_typedefs.get(category, [])
        names = [d["name"] for d in defs[:10]]
        print(f"  {category} ({len(defs)}): {', '.join(names)}{'...' if len(defs) > 10 else ''}")
except Exception as e:
    print(f"Blad: {e}")
    print("Przykladowe typy wbudowane w Atlas:")
    print("  entityDefs: DataSet, Process, Infrastructure, hdfs_path, hive_table, spark_process")
    print("  classificationDefs: PII, Confidential, Sensitive")

In [ ]:
# Definiowanie wlasnych typow dla systemu rekomendacji

# 1. Enum - poziomy jakosci danych
data_quality_enum = {
    "enumDefs": [{
        "name": "data_quality_level",
        "description": "Poziom jakosci danych w Medallion Architecture",
        "elementDefs": [
            {"value": "BRONZE", "ordinal": 0},
            {"value": "SILVER", "ordinal": 1},
            {"value": "GOLD", "ordinal": 2}
        ]
    }]
}

# 2. Entity - typ dla datasetu rekomendacji
recommender_types = {
    "entityDefs": [
        {
            "name": "recommender_dataset",
            "description": "Dataset w systemie rekomendacji MovieLens",
            "superTypes": ["DataSet"],
            "attributeDefs": [
                {"name": "storage_location", "typeName": "string", "isOptional": True},
                {"name": "file_format", "typeName": "string", "isOptional": True},
                {"name": "quality_level", "typeName": "data_quality_level", "isOptional": True},
                {"name": "row_count", "typeName": "long", "isOptional": True},
                {"name": "schema_version", "typeName": "string", "isOptional": True},
                {"name": "last_updated", "typeName": "date", "isOptional": True}
            ]
        },
        {
            "name": "recommender_etl_process",
            "description": "Proces ETL w systemie rekomendacji",
            "superTypes": ["Process"],
            "attributeDefs": [
                {"name": "technology", "typeName": "string", "isOptional": True},
                {"name": "schedule", "typeName": "string", "isOptional": True},
                {"name": "transformation_logic", "typeName": "string", "isOptional": True},
                {"name": "spark_app_id", "typeName": "string", "isOptional": True}
            ]
        }
    ]
}

try:
    # Najpierw enum
    result = atlas.post("/types/typedefs", data_quality_enum)
    print(f"Utworzono enum: data_quality_level")

    # Potem entity types
    result = atlas.post("/types/typedefs", recommender_types)
    print(f"Utworzono typy: recommender_dataset, recommender_etl_process")
except Exception as e:
    print(f"Blad lub typy juz istnieja: {e}")

# Wyswietl definicje typow
print("\nDefinicja recommender_dataset:")
print(json.dumps(recommender_types["entityDefs"][0], indent=2))

## 4. Rejestrowanie encji - datasety MovieLens

Kazdy dataset w naszym systemie rekomendacji powinien byc zarejestrowany w Atlas jako encja.
Dzieki temu mamy centralny katalog wszystkich danych.

In [ ]:
def create_dataset_entity(name, qualified_name, description, storage_location,
                          file_format, quality_level, row_count=None, columns=None):
    """Tworzy encje datasetu w Atlas."""
    entity = {
        "entity": {
            "typeName": "recommender_dataset",
            "attributes": {
                "name": name,
                "qualifiedName": qualified_name,
                "description": description,
                "storage_location": storage_location,
                "file_format": file_format,
                "quality_level": quality_level,
                "row_count": row_count,
                "owner": "data_engineering_team"
            }
        }
    }
    return entity


def create_process_entity(name, qualified_name, description, technology,
                          inputs, outputs, transformation_logic=None):
    """Tworzy encje procesu ETL w Atlas."""
    entity = {
        "entity": {
            "typeName": "recommender_etl_process",
            "attributes": {
                "name": name,
                "qualifiedName": qualified_name,
                "description": description,
                "technology": technology,
                "transformation_logic": transformation_logic,
                "inputs": inputs,
                "outputs": outputs,
                "owner": "data_engineering_team"
            }
        }
    }
    return entity


# Rejestracja datasetow Bronze
datasets = [
    # Bronze Layer
    create_dataset_entity(
        name="ratings_bronze",
        qualified_name="recommender.bronze.ratings@movielens",
        description="Surowe dane ocen uzytkownikow z MovieLens (warstwa Bronze)",
        storage_location="hdfs://namenode:9000/data/movielens/bronze/ratings",
        file_format="parquet",
        quality_level="BRONZE",
        row_count=25000095
    ),
    create_dataset_entity(
        name="movies_bronze",
        qualified_name="recommender.bronze.movies@movielens",
        description="Surowe dane filmow z MovieLens (warstwa Bronze)",
        storage_location="hdfs://namenode:9000/data/movielens/bronze/movies",
        file_format="parquet",
        quality_level="BRONZE",
        row_count=62423
    ),
    # Silver Layer
    create_dataset_entity(
        name="ratings_silver",
        qualified_name="recommender.silver.ratings@movielens",
        description="Oczyszczone oceny - deduplikacja, walidacja, enrichment",
        storage_location="hdfs://namenode:9000/data/movielens/silver/ratings",
        file_format="parquet",
        quality_level="SILVER",
        row_count=25000000
    ),
    create_dataset_entity(
        name="movies_silver",
        qualified_name="recommender.silver.movies@movielens",
        description="Oczyszczone dane filmow - parsowanie roku, gatunkow, enrichment",
        storage_location="hdfs://namenode:9000/data/movielens/silver/movies",
        file_format="parquet",
        quality_level="SILVER",
        row_count=62423
    ),
    # Gold Layer
    create_dataset_entity(
        name="movie_stats_gold",
        qualified_name="recommender.gold.movie_stats@movielens",
        description="Agregaty statystyk filmow - avg rating, wilson score, popularnosc",
        storage_location="hdfs://namenode:9000/data/movielens/gold/movie_stats",
        file_format="parquet",
        quality_level="GOLD"
    ),
    create_dataset_entity(
        name="user_profiles_gold",
        qualified_name="recommender.gold.user_profiles@movielens",
        description="Profile uzytkownikow - segmentacja, preferencje, aktywnosc",
        storage_location="hdfs://namenode:9000/data/movielens/gold/user_profiles",
        file_format="parquet",
        quality_level="GOLD"
    ),
    # PostgreSQL source
    create_dataset_entity(
        name="pg_ratings",
        qualified_name="recommender.postgresql.movielens.ratings@movielens",
        description="Tabela zrodlowa ratings w PostgreSQL",
        storage_location="jdbc:postgresql://postgres:5432/recommender#movielens.ratings",
        file_format="jdbc",
        quality_level="BRONZE",
        row_count=25000095
    ),
    create_dataset_entity(
        name="pg_movies",
        qualified_name="recommender.postgresql.movielens.movies@movielens",
        description="Tabela zrodlowa movies w PostgreSQL",
        storage_location="jdbc:postgresql://postgres:5432/recommender#movielens.movies",
        file_format="jdbc",
        quality_level="BRONZE",
        row_count=62423
    )
]

# Rejestracja w Atlas
registered_guids = {}
for ds in datasets:
    name = ds["entity"]["attributes"]["name"]
    try:
        result = atlas.post("/entity", ds)
        guid = result.get("guidAssignments", {}).get(list(result.get("guidAssignments", {}).keys())[0], "")
        registered_guids[name] = guid
        print(f"Zarejestrowano: {name} (GUID: {guid[:8]}...)")
    except Exception as e:
        print(f"Blad rejestracji {name}: {e}")

print(f"\nZarejestrowano {len(registered_guids)} datasetow")

## 5. Klasyfikacja danych - PII, Confidential, Public

Klasyfikacje (tagi) w Atlas pozwalaja oznaczac dane pod katem:
- **PII** (Personally Identifiable Information) - dane osobowe
- **Confidential** - dane poufne
- **Public** - dane publiczne

Klasyfikacje propaguja sie automatycznie przez lineage - jesli zrodlo jest PII, to wynik transformacji tez jest PII.

In [ ]:
# Definiowanie klasyfikacji
classifications_def = {
    "classificationDefs": [
        {
            "name": "PII",
            "description": "Dane osobowe (Personally Identifiable Information)",
            "attributeDefs": [
                {"name": "pii_type", "typeName": "string", "isOptional": True},
                {"name": "gdpr_relevant", "typeName": "boolean", "isOptional": True}
            ],
            "superTypes": []
        },
        {
            "name": "Confidential",
            "description": "Dane poufne - ograniczony dostep",
            "attributeDefs": [
                {"name": "sensitivity_level", "typeName": "string", "isOptional": True}
            ],
            "superTypes": []
        },
        {
            "name": "Public",
            "description": "Dane publiczne - wolny dostep",
            "attributeDefs": [],
            "superTypes": []
        },
        {
            "name": "ML_Feature",
            "description": "Dane uzywaane jako feature w modelach ML",
            "attributeDefs": [
                {"name": "model_name", "typeName": "string", "isOptional": True},
                {"name": "feature_importance", "typeName": "float", "isOptional": True}
            ],
            "superTypes": []
        }
    ]
}

try:
    atlas.post("/types/typedefs", classifications_def)
    print("Utworzono klasyfikacje: PII, Confidential, Public, ML_Feature")
except Exception as e:
    print(f"Blad: {e}")

# Przypisanie klasyfikacji do datasetow
# Ratings zawieraja user_id - to PII!
pii_classification = {
    "classification": {
        "typeName": "PII",
        "attributes": {
            "pii_type": "user_identifier",
            "gdpr_relevant": True
        }
    },
    "entityGuids": []  # Tutaj wstawiamy GUIDy encji ratings
}

# Przyklad: klasyfikacja encji po qualifiedName
def classify_entity(qualified_name, classification_name, attributes=None):
    """Nadaje klasyfikacje encji na podstawie qualifiedName."""
    try:
        # Znajdz encje
        search_result = atlas.get("/entity/uniqueAttribute/type/recommender_dataset",
                                  params={"attr:qualifiedName": qualified_name})
        guid = search_result["entity"]["guid"]

        # Przypisz klasyfikacje
        classification = {"typeName": classification_name}
        if attributes:
            classification["attributes"] = attributes

        atlas.post(f"/entity/guid/{guid}/classifications", [classification])
        print(f"Klasyfikacja {classification_name} nadana: {qualified_name}")
    except Exception as e:
        print(f"Blad klasyfikacji: {e}")


# Klasyfikuj datasety
classify_entity("recommender.bronze.ratings@movielens", "PII",
                {"pii_type": "user_identifier", "gdpr_relevant": True})
classify_entity("recommender.silver.ratings@movielens", "PII",
                {"pii_type": "user_identifier", "gdpr_relevant": True})
classify_entity("recommender.bronze.movies@movielens", "Public")
classify_entity("recommender.silver.movies@movielens", "Public")
classify_entity("recommender.gold.movie_stats@movielens", "Public")
classify_entity("recommender.gold.user_profiles@movielens", "Confidential",
                {"sensitivity_level": "high"})
classify_entity("recommender.gold.user_profiles@movielens", "ML_Feature",
                {"model_name": "ALS_recommender"})

## 6. Data Lineage - sledzenie transformacji

Lineage to graf pokazujacy:
- **Skad** dane pochodza (upstream)
- **Dokad** ida (downstream)
- **Jak** sa transformowane (procesy)

```
PostgreSQL            Bronze              Silver              Gold
+-----------+     +------------+      +------------+     +-------------+
| pg_ratings| --> | ratings_   | -->  | ratings_   | --> | movie_stats |
|           |     | bronze     |      | silver     |     |             |
+-----------+     +------------+      +------------+     +-------------+
                                           |             +-------------+
+-----------+     +------------+      +----+-----+  -->  | user_       |
| pg_movies | --> | movies_    | -->  | movies_  |       | profiles    |
|           |     | bronze     |      | silver   |       +-------------+
+-----------+     +------------+      +----------+
```

In [ ]:
def entity_ref(qualified_name, type_name="recommender_dataset"):
    """Tworzy referencje do encji Atlas."""
    return {"typeName": type_name, "uniqueAttributes": {"qualifiedName": qualified_name}}


# Rejestracja procesow ETL (tworzacych lineage)
etl_processes = [
    # PostgreSQL -> Bronze
    create_process_entity(
        name="ingest_ratings_pg_to_bronze",
        qualified_name="recommender.etl.ingest_ratings_pg_to_bronze@movielens",
        description="Ingestia ratings z PostgreSQL do warstwy Bronze na HDFS",
        technology="PySpark JDBC",
        inputs=[entity_ref("recommender.postgresql.movielens.ratings@movielens")],
        outputs=[entity_ref("recommender.bronze.ratings@movielens")],
        transformation_logic="spark.read.jdbc() -> write.parquet() + metadane ingestii"
    ),
    create_process_entity(
        name="ingest_movies_pg_to_bronze",
        qualified_name="recommender.etl.ingest_movies_pg_to_bronze@movielens",
        description="Ingestia movies z PostgreSQL do warstwy Bronze na HDFS",
        technology="PySpark JDBC",
        inputs=[entity_ref("recommender.postgresql.movielens.movies@movielens")],
        outputs=[entity_ref("recommender.bronze.movies@movielens")],
        transformation_logic="spark.read.jdbc() -> write.parquet() + metadane ingestii"
    ),
    # Bronze -> Silver
    create_process_entity(
        name="clean_ratings_bronze_to_silver",
        qualified_name="recommender.etl.clean_ratings@movielens",
        description="Czyszczenie ratings: deduplikacja, walidacja, enrichment",
        technology="PySpark",
        inputs=[entity_ref("recommender.bronze.ratings@movielens")],
        outputs=[entity_ref("recommender.silver.ratings@movielens")],
        transformation_logic="dropDuplicates + filter(rating 0.5-5.0) + date enrichment"
    ),
    create_process_entity(
        name="clean_movies_bronze_to_silver",
        qualified_name="recommender.etl.clean_movies@movielens",
        description="Czyszczenie movies: parsowanie roku, gatunkow, enrichment",
        technology="PySpark",
        inputs=[entity_ref("recommender.bronze.movies@movielens")],
        outputs=[entity_ref("recommender.silver.movies@movielens")],
        transformation_logic="regexp_extract(year) + split(genres) + clean_title"
    ),
    # Silver -> Gold
    create_process_entity(
        name="aggregate_movie_stats",
        qualified_name="recommender.etl.aggregate_movie_stats@movielens",
        description="Agregacja statystyk filmow: avg rating, wilson score, popularnosc",
        technology="PySpark",
        inputs=[
            entity_ref("recommender.silver.ratings@movielens"),
            entity_ref("recommender.silver.movies@movielens")
        ],
        outputs=[entity_ref("recommender.gold.movie_stats@movielens")],
        transformation_logic="groupBy(movie_id) + agg(avg, count, stddev) + wilson_score + join(movies)"
    ),
    create_process_entity(
        name="build_user_profiles",
        qualified_name="recommender.etl.build_user_profiles@movielens",
        description="Budowanie profili uzytkownikow: segmentacja, preferencje",
        technology="PySpark",
        inputs=[entity_ref("recommender.silver.ratings@movielens")],
        outputs=[entity_ref("recommender.gold.user_profiles@movielens")],
        transformation_logic="groupBy(user_id) + agg(stats) + user_segment classification"
    )
]

for proc in etl_processes:
    name = proc["entity"]["attributes"]["name"]
    try:
        result = atlas.post("/entity", proc)
        print(f"Zarejestrowano proces: {name}")
    except Exception as e:
        print(f"Blad: {name} - {e}")

print("\nLineage zarejestrowany! Sprawdz w Atlas UI: http://atlas:21000")

In [ ]:
# Odczyt lineage z API
def get_lineage(qualified_name, direction="BOTH", depth=10):
    """Pobiera lineage encji."""
    try:
        entity = atlas.get("/entity/uniqueAttribute/type/recommender_dataset",
                           params={"attr:qualifiedName": qualified_name})
        guid = entity["entity"]["guid"]

        lineage = atlas.get(f"/lineage/{guid}",
                            params={"direction": direction, "depth": depth})
        return lineage
    except Exception as e:
        print(f"Blad: {e}")
        return None


def print_lineage(lineage):
    """Wyswietla lineage w czytelny sposob."""
    if not lineage:
        return

    guid_map = lineage.get("guidEntityMap", {})
    relations = lineage.get("relations", [])

    print("\n--- Encje w lineage ---")
    for guid, entity in guid_map.items():
        name = entity.get("displayText", "unknown")
        type_name = entity.get("typeName", "unknown")
        print(f"  [{type_name}] {name}")

    print("\n--- Relacje ---")
    for rel in relations:
        from_name = guid_map.get(rel["fromEntityId"], {}).get("displayText", "?")
        to_name = guid_map.get(rel["toEntityId"], {}).get("displayText", "?")
        print(f"  {from_name} --> {to_name}")


# Lineage dla movie_stats (Gold)
print("=" * 60)
print("Lineage dla: movie_stats_gold")
print("=" * 60)
lineage = get_lineage("recommender.gold.movie_stats@movielens", direction="INPUT")
print_lineage(lineage)

print("\n" + "=" * 60)
print("Lineage dla: ratings_silver (upstream + downstream)")
print("=" * 60)
lineage = get_lineage("recommender.silver.ratings@movielens", direction="BOTH")
print_lineage(lineage)

## 7. Glossary biznesowy

Glossary to slownik terminow biznesowych. Pozwala:
- Zdefiniowac jednoznaczne definicje pojec
- Powiazac terminy z konkretnymi encjami danych
- Zorganizowac terminy w kategorie

Przyklad: co dokladnie znaczy "rating" w naszym systemie? Skala 0.5-5.0, pol-gwiazdkowe kroki.

In [ ]:
# Tworzenie glossary
glossary = {
    "name": "MovieLens Recommender Glossary",
    "shortDescription": "Slownik terminow systemu rekomendacji MovieLens",
    "longDescription": "Centralny slownik definiujacy pojecia biznesowe uzywane "
                        "w systemie rekomendacji filmow opartym na danych MovieLens 25M."
}

try:
    glossary_result = atlas.post("/glossary", glossary)
    glossary_guid = glossary_result["guid"]
    print(f"Utworzono glossary: {glossary_result['name']} (GUID: {glossary_guid[:8]}...)")
except Exception as e:
    print(f"Blad: {e}")
    glossary_guid = None

# Definiowanie terminow
terms = [
    {
        "name": "User",
        "shortDescription": "Uzytkownik systemu rekomendacji",
        "longDescription": "Osoba korzystajaca z systemu, identyfikowana przez user_id. "
                           "Kazdy uzytkownik moze oceniac filmy (ratings) i otrzymywac rekomendacje. "
                           "User_id jest danym PII i podlega ochronie GDPR.",
        "anchor": {"glossaryGuid": glossary_guid}
    },
    {
        "name": "Rating",
        "shortDescription": "Ocena filmu przez uzytkownika",
        "longDescription": "Numeryczna ocena filmu w skali 0.5 do 5.0 z krokiem 0.5. "
                           "Rating >= 4.0 jest uznawany za pozytywny (is_positive). "
                           "Kazdy uzytkownik moze ocenic dany film tylko raz.",
        "anchor": {"glossaryGuid": glossary_guid}
    },
    {
        "name": "Movie",
        "shortDescription": "Film w katalogu MovieLens",
        "longDescription": "Film identyfikowany przez movie_id. Zawiera tytul, rok produkcji "
                           "i liste gatunkow. Dane filmowe sa publiczne.",
        "anchor": {"glossaryGuid": glossary_guid}
    },
    {
        "name": "Recommendation",
        "shortDescription": "Rekomendacja filmu dla uzytkownika",
        "longDescription": "Wynik modelu ML (ALS) - przewidywana ocena filmu przez uzytkownika. "
                           "Rekomendacje sa generowane na podstawie profilu uzytkownika "
                           "i historycznych ocen podobnych uzytkownikow (collaborative filtering).",
        "anchor": {"glossaryGuid": glossary_guid}
    },
    {
        "name": "Wilson Score",
        "shortDescription": "Metryka rankingowa filmow",
        "longDescription": "Lower bound of Wilson score confidence interval. "
                           "Balansuje srednia ocene z liczba ocen - film z 4.8 srednia "
                           "ale tylko 3 ocenami bedzie nizej niz film z 4.5 i 1000 ocen. "
                           "Uzywany w warstwie Gold do rankingu filmow.",
        "anchor": {"glossaryGuid": glossary_guid}
    },
    {
        "name": "User Segment",
        "shortDescription": "Segment uzytkownika",
        "longDescription": "Klasyfikacja uzytkownika na podstawie aktywnosci: "
                           "power_user (>=1000 ocen), active (>=100), casual (>=20), new (<20). "
                           "Uzywany w warstwie Gold user_profiles.",
        "anchor": {"glossaryGuid": glossary_guid}
    }
]

for term in terms:
    try:
        result = atlas.post("/glossary/term", term)
        print(f"Utworzono termin: {term['name']}")
    except Exception as e:
        print(f"Blad tworzenia terminu {term['name']}: {e}")

print("\nGlossary dostepny w Atlas UI -> Glossary tab")

## 8. Atlas + Spark Hook - automatyczne sledzenie

Spark Atlas Hook automatycznie rejestruje:
- Spark aplikacje jako Process
- Odczytane/zapisane datasety jako Entity
- Lineage miedzy nimi

Konfiguracja wymaga dodania hook JAR do Spark i ustawienia `atlas-application.properties`.

```properties
# atlas-application.properties (na Spark driver/executor)
atlas.rest.address=http://atlas:21000
atlas.hook.spark.enabled=true
atlas.cluster.name=movielens
```

```bash
# spark-defaults.conf
spark.extraListeners=com.hortonworks.spark.atlas.SparkAtlasEventTracker
spark.sql.queryExecutionListeners=com.hortonworks.spark.atlas.SparkAtlasEventTracker
```

In [ ]:
# Symulacja tego co Spark Hook rejestruje automatycznie
# W produkcji hook robi to sam - tutaj pokazujemy strukture

spark_job_entity = {
    "entity": {
        "typeName": "spark_process",
        "attributes": {
            "name": "ALS_Model_Training",
            "qualifiedName": "recommender.spark.als_training_20250101@movielens",
            "description": "Trening modelu ALS na danych MovieLens",
            # Spark Hook automatycznie wypelnia ponizsze:
            "inputs": [
                entity_ref("recommender.silver.ratings@movielens")
            ],
            "outputs": [
                entity_ref("recommender.gold.user_profiles@movielens")
            ],
            "sparkPlanDescription": "ALS.fit(ratings) -> model.save()",
            "owner": "spark_hook_auto"
        }
    }
}

print("Spark Hook automatycznie rejestruje:")
print(json.dumps(spark_job_entity, indent=2))
print("\nKorzysci Spark Hook:")
print("  1. Automatyczny lineage - nie trzeba recznie rejestrowac")
print("  2. Pelna historia jobow Spark")
print("  3. Powiazanie z Spark UI (app_id)")
print("  4. Sledzenie schema evolution")

## 9. Wyszukiwanie - DSL queries i full-text search

Atlas oferuje dwa sposoby wyszukiwania:
- **Basic search**: filtrowanie po typie, klasyfikacji, atrybutach
- **DSL search**: Atlas Query Language (podobny do SQL)
- **Full-text search**: wyszukiwanie pelnotekstowe (Apache Solr)

In [ ]:
def search_basic(type_name=None, classification=None, query=None, limit=10):
    """Wyszukiwanie podstawowe w Atlas."""
    params = {"limit": limit}
    if type_name:
        params["typeName"] = type_name
    if classification:
        params["classification"] = classification
    if query:
        params["query"] = query

    try:
        result = atlas.get("/search/basic", params=params)
        entities = result.get("entities", [])
        print(f"Znaleziono {len(entities)} wynikow:")
        for e in entities:
            name = e.get("attributes", {}).get("name", e.get("displayText", "?"))
            type_n = e.get("typeName", "?")
            classifications = [c["typeName"] for c in e.get("classifications", [])]
            print(f"  [{type_n}] {name} {classifications if classifications else ''}")
        return entities
    except Exception as e:
        print(f"Blad: {e}")
        return []


def search_dsl(query, limit=10):
    """Wyszukiwanie DSL w Atlas."""
    try:
        result = atlas.get("/search/dsl", params={"query": query, "limit": limit})
        entities = result.get("entities", [])
        print(f"DSL query: {query}")
        print(f"Znaleziono: {len(entities)} wynikow")
        for e in entities:
            print(f"  - {e.get('displayText', '?')} ({e.get('typeName', '?')})")
        return entities
    except Exception as e:
        print(f"Blad: {e}")
        return []


# Przyklady wyszukiwan
print("=== 1. Wszystkie datasety rekomendacji ===")
search_basic(type_name="recommender_dataset")

print("\n=== 2. Datasety z klasyfikacja PII ===")
search_basic(classification="PII")

print("\n=== 3. Wyszukiwanie full-text: 'ratings' ===")
search_basic(query="ratings")

print("\n=== 4. DSL: datasety w warstwie Gold ===")
search_dsl("recommender_dataset where quality_level = 'GOLD'")

print("\n=== 5. DSL: datasety z wiecej niz milion wierszy ===")
search_dsl("recommender_dataset where row_count > 1000000")

## Zadanie koncowe

Zarejestruj pelny lineage systemu rekomendacji MovieLens w Atlas:

1. **Dodatkowe datasety**: zarejestruj `genre_trends_gold`, `recommendations_output`, `als_model`
2. **Procesy**: zarejestruj `train_als_model`, `generate_recommendations`, `export_to_api`
3. **Klasyfikacje**: oznacz `recommendations_output` jako `ML_Feature` i `Confidential`
4. **Glossary**: dodaj terminy: `Genre`, `Collaborative Filtering`, `Positive Rate`
5. **Wyszukiwanie**: napisz DSL query ktore znajdzie wszystkie datasety PII w warstwie Silver
6. **Lineage**: pobierz i wyswietl pelny lineage od PostgreSQL do recommendations_output

Cel: kazdy dataset i transformacja w systemie jest udokumentowana, sklasyfikowana i sledzna.

In [ ]:
# Twoje rozwiazanie:


In [ ]:
# Podsumowanie: co daje Atlas w systemie rekomendacji?
print("""
=== Podsumowanie: Apache Atlas w systemie rekomendacji ===

1. KATALOG DANYCH
   - Centralny rejestr wszystkich datasetow (PostgreSQL, HDFS, Gold tables)
   - Metadane: schema, rozmiar, wlasciciel, opis

2. KLASYFIKACJA
   - PII: dane uzytkownikow (user_id, ratings) -> wymuszenie GDPR
   - Public: dane filmow -> swobodny dostep
   - ML_Feature: dane uzywane w modelach -> sledzenie model lineage

3. LINEAGE
   - Pelna sciezka: PostgreSQL -> Bronze -> Silver -> Gold -> API
   - Impact analysis: zmiana w ratings -> co jeszcze sie zmieni?
   - Root cause: blad w movie_stats -> skad dane przychodza?

4. GLOSSARY
   - Jednoznaczne definicje (co to "rating", "user_segment")
   - Wspolny jezyk miedzy data engineering a biznesem

5. WYSZUKIWANIE
   - "Gdzie sa dane PII?" -> lista wszystkich datasetow z PII
   - "Jakie dane zasila model ALS?" -> lineage query

Atlas integruje sie z Ranger (notebook 26) do automatycznego
wymuszania polityk bezpieczenstwa na podstawie klasyfikacji.
""")